In [837]:
import numpy as np

# US

In [838]:
sigma  = 2
sigmah = 3
psi_us = [0.15,1]
y_us   = 1

alpha0ref = psi_us[1]/psi_us[0]
phiref    = ( 1/( alpha0ref) )*( ((1-psi_us[0])**(-sigma))/ (psi_us[1]**(-sigmah) ) ) 
alpha0ref, phiref

(6.666666666666667, 0.20761245674740486)

# EU

In [839]:
y_eu   = 0.78639*y_us 
psi_eu = [0.096,1.047]

p_eu      = alpha0ref*phiref*(y_eu**sigma)*( psi_eu[1]**(-sigmah) )*( (1-psi_eu[0])**sigma ) 
alpha1_eu = psi_eu[1] - alpha0ref*(y_eu/p_eu)*psi_eu[0]
p_eu, alpha1_eu

(0.6094455761530171, 0.2211845052401592)

# price decomposition

$$ \widehat{p} = 
\frac{-\chi(1-\Psi_1) \left(\sigma \frac{1}{\alpha_0} \frac{\Psi_2}{\Psi_1} \frac{p}{y} - \sigma_h\right) + \sigma }{\chi (1-\Psi_1) \left(\sigma_h - \frac{1}{\alpha_0} \frac{\Psi_2}{\Psi_1} \frac{p}{y} \right) + 1 + \Psi_1(\sigma-1)}\widehat{y}  +  \frac{\chi\sigma_h \frac{1-\Psi_1}{\Psi_1} \frac{\alpha_1}{\alpha_0}\frac{p}{y} + \sigma \frac{\alpha_1}{\alpha_0}\frac{p}{y}}{\chi (1-\Psi_1) \left(\sigma_h - \frac{1}{\alpha_0} \frac{\Psi_2}{\Psi_1} \frac{p}{y} \right) + 1 + \Psi_1(\sigma-1)}\widehat{\alpha}_1 
$$ 
given that 
$$
\left.\begin{array}{rcl}
\widehat{\Psi}_1 &= &\widehat{p} + \widehat{m} - \widehat{y}\\
\widehat{\Psi}_2 &=&  \alpha_0 \frac{\Psi_1}{\Psi_2} \frac{y}{p} \widehat{m} + \frac{\alpha_1}{\Psi_2} \widehat{\alpha}_1 \\
\widehat{\Psi}_2 &=& \chi \widehat{\Psi}_1
\end{array}\right\} ~~\Rightarrow~~\widehat{m} = -\frac{\chi}{\chi - \alpha_0 \frac{\Psi_1}{\Psi_2} \frac{y}{p}} \widehat{p} + \frac{\chi}{\chi - \alpha_0 \frac{\Psi_1}{\Psi_2} \frac{y}{p}} \widehat{y}  + \frac{\frac{\alpha_1}{\Psi_2}}{\chi - \alpha_0 \frac{\Psi_1}{\Psi_2} \frac{y}{p}} \widehat{\alpha}_1
$$

In [840]:
chi_obs = ((psi_us[1]-psi_eu[1])/psi_eu[1]) / ((psi_us[0]-psi_eu[0])/psi_eu[0])

chi=-.13

Num1 = -chi*(1-psi_eu[0])*(sigma*(1/alpha0ref)*(psi_eu[1]/psi_eu[0])*(p_eu/y_eu) - sigmah) + sigma
Num2 = chi*sigmah*((1-psi_eu[0])/psi_eu[0])*(alpha1_eu/alpha0ref)*(p_eu/y_eu) + sigma*(alpha1_eu/alpha0ref)*(p_eu/y_eu)
Den  = chi*(1-psi_eu[0])*(sigmah-(1/alpha0ref)*(psi_eu[1]/psi_eu[0])*(p_eu/y_eu)) + 1+ psi_eu[0]*(sigma-1)

Coeff1 = Num1/Den
Coeff2 = Num2/Den

Coeff1, Coeff2, chi, chi_obs

(2.179912003664755, -0.04818721345126506, -0.13, -0.07980473309986194)

In [841]:
phat = Coeff1*((y_us-y_eu)/y_eu) + Coeff2*(-1)
phat

0.6403247064290856

In [842]:
Coeff1*((y_us-y_eu)/y_eu)/phat

0.9247456595576455

In [843]:
Coeff2*(-1)/phat

0.07525434044235443

In [844]:
(1-p_eu)/p_eu

0.6408356039144077

# Identification

The model is
$$
\max_{m_i} \frac{(y_i-pm_i)^{1-\sigma}}{1-\sigma} + \phi (1-\exp(-\alpha_1 - \alpha_0 m_i))
$$
The FOC is 
$$
p (y_i-pm_i)^{-\sigma} = \phi\alpha_0 \exp(-\alpha_1 - \alpha_0 m_i)
$$
This gives a solution $m_i = {\cal M}(y_i,p|\sigma, \phi, \alpha_0, \alpha1)$


### US
For the US, we have $p=1$ and $\sum_i \omega_i y_i \equiv y = 1$. Therefore, the moments restrictions  
$$
\Psi_1 = \frac{p \sum_i \omega_i m_i}{\sum_i \omega_i y_i}
$$
$$
\Psi_2 = \sum_i (1-\exp(-\alpha_1 - \alpha_0 m_i))
$$
$$
\Psi_3 = \frac{(1-\exp(-\alpha_1 - \alpha_0 m_{\max}))}{(1-\exp(-\alpha_1 - \alpha_0 m_{\min}))}
$$
allows to identify $\{\phi,\alpha_0,\alpha_1\}$


### Europe

For Europe, we keep $\{\phi,\alpha_0,\alpha_1\}$ found in the previous step, and the moments restrictions
$$
\Psi_1 = \frac{p \sum_i \omega_i m_i}{\sum_i \omega_i y_i}
$$
$$
\Psi_2 = \sum_i (1-\exp(-\alpha_1 - \alpha_0 m_i))
$$
allows to identify $\{p,\alpha_1\}$, given that a calibration of $\sum_i \omega_i y_i \equiv y$



In [905]:
from scipy.optimize import fsolve, bisect, brentq

## Calibrated parameter

In [906]:
sigma  = 2

## Aggregate moments

Here we use the average of the gradient in EU and the US gradient for the 4th quartile relative to the first. 

In [907]:
grad_eu = 1.061
grad_us = 1.276
grad_eu

1.061

Next, we setup the moments. Note that I used the average health from the ADL definition in Table 4. This yields an average of 0.89 in US and 0.93 in Europe.  

In [908]:
y_us   = 1
psi_us = [0.143,0.892,grad_us]
y_eu   = 0.783*y_us 
psi_eu = [0.087,0.931,grad_eu]

## Income inequalities: the US

Francois was using external data. Instead, consider Table 5. It gives the variance of the income process. It is already presented in the paper. If log income has mean 0 and variance equal to $\sigma^2_{\epsilon}$, we can solve for the quartiles (we use mid-points of quartiles, 12.5, 37.5, 62.5 and 87.5)

In [909]:
from scipy.stats import norm

We take the mean for EU in Table 5. 

In [910]:
sig_us = 0.493
sig_eu = np.mean([0.26,0.0956,0.2367,0.275,0.181,0.0956,0.2776])

We find the percentiles, transform in levels and normalize to 1. 

In [911]:
#qs = [0.125,0.375,0.625,0.875]
#yy_us = [norm(0,np.sqrt(sig_us)).ppf(q) for q in qs]
#yy_us = np.exp(yy_us)
#yy_us = yy_us/np.mean(yy_us)
#yy_us

In [912]:
yy_us = np.array([0.209,0.521,0.930,2.339])
yy_eu = np.array([0.290,0.632,0.992,2.086])

The mean is one by construction

In [913]:
np.mean(yy_us)

0.99975

We do the same for EU,

In [914]:
#yy_eu = [norm(0,np.sqrt(sig_eu)).ppf(q) for q in qs]
#yy_eu = np.exp(yy_eu)
#yy_eu = yy_eu/np.mean(yy_eu)
#yy_eu

In [915]:
np.mean(yy_eu)

1.0

We reduce EU income using our relative income...

In [916]:
yy_eu = y_eu*yy_eu

In [917]:
yy_eu

array([0.22707 , 0.494856, 0.776736, 1.633338])

In [918]:
np.mean(yy_eu)

0.7829999999999999

Here are relative incomes. Much more inequality in US

In [919]:
yy_us[-1]/yy_us[0], yy_eu[-1]/yy_eu[0]

(11.191387559808613, 7.1931034482758625)

## Function for finding the decision rule for $m$

I did not check this function. I presume it is ok. 

In [920]:
# Function to solve m
def medexp(m, *param_m):
    p, y, sigma, alpha0, phi, alpha1 = param_m 
    criterion = p*( (y - p*max(0,min(m,y/p)))**(-sigma) ) - alpha0*phi*np.exp(-( alpha1 + alpha0*max(0,min(m,y/p)) ))
    return  criterion

## Function for finding parameters $\{\alpha_0, \phi,\alpha_1\}$ using US moments' restrictions

Here I did a few changes because the wrong incomes were used. Let's use yy in the arguments directly. I also used the mean function which allows to avoid the hard coding of 5 (quintiles) before.  

In [921]:
def hetero_calib(pd_c, *param_c):
    Psi1, Psi2, Psi3, p, yy, sigma = param_c 
    msolv = np.zeros(yy.size)
    for ii in range(yy.size):
        param_cm  = (p, yy[ii], sigma, pd_c[0], pd_c[1], pd_c[2])
        m0        = .1
        msolv[ii] = max(0,bisect(medexp, m0, args=param_cm))
    
    return [Psi1 - np.mean(msolv), 
            Psi2 - np.mean(1-np.exp(-pd_c[2]-pd_c[0]*msolv)),
            Psi3 - (1-np.exp(-pd_c[2]-pd_c[0]*msolv[-1]))/(1-np.exp(-pd_c[2]-pd_c[0]*msolv[0])) ] 

In [926]:
def hetero_calib_f(pd_c, *param_c):
	Psi1, Psi2, Psi3, p, yy, sigma = param_c 
	msolv = np.zeros(yy.size)
	for ii in range(yy.size):
		param_cm  = (p, yy[ii], sigma, pd_c[0], pd_c[1], pd_c[2])
		msolv[ii] = max(0,brentq(medexp, a=0.01,b=0.99*yy[ii], args=param_cm))
		print(yy[ii],msolv[ii])
	mms = [Psi1 - np.mean(msolv), 
		Psi2 - np.mean(1-np.exp(-pd_c[2]-pd_c[0]*msolv)),
		Psi3 - (1-np.exp(-pd_c[2]-pd_c[0]*msolv[-1]))/(1-np.exp(-pd_c[2]-pd_c[0]*msolv[0])) ]
	criterion = 0.0
	for m in mms:
		criterion += (m**2)
	return criterion

## Solution for the US

In [927]:
# Benchmark US
#param_c   = (psi_us[0], psi_us[1], psi_us[2], 1, yy_us, sigma)
#pd_c0     = [5 , 4 , 1.5 ]
#calibsol  = fsolve(hetero_calib, pd_c0, args=param_c)#, full_output=True)
#calibsol

In [928]:
from scipy.optimize import minimize, bisect

In [929]:
# Benchmark US
param_c   = (psi_us[0], psi_us[1], psi_us[2], 1, yy_us, sigma)
pd_c0     = [5 , 4 , 1.5 ]
calibsol  = minimize(hetero_calib_f, pd_c0,args=param_c,method='powell')
calibsol.x

ValueError: f(a) and f(b) must have different signs

## Function for finding $\{p, \alpha_1\}$ using European moments' restrictions

Here, there was a mistake. US incomes were used. We now use yy, internal to the function. 

In [ ]:
def hetero_calib_eu(pd_s, *param_s):
    
    Psi1, Psi2, yy, sigma, phi, alpha0 = param_s 
    
    msolv = np.zeros(yy.size)
    
    for ii in range(yy.size):
        param_sm  = (pd_s[0], yy[ii], sigma, alpha0, phi, pd_s[1])
        m0        = .1
        msolv[ii] = max(0,bisect(medexp, a=0.0,b=0.9, args=param_sm))
    
    return [Psi1 - (pd_s[0]*np.mean(msolv))/(np.mean(yy)), 
            Psi2 - np.mean(1-np.exp(-pd_s[1]-alpha0*msolv))] 

In [ ]:
def hetero_calib_eu_f(pd_s, *param_s):
    
	Psi1, Psi2, yy, sigma, phi, alpha0 = param_s 

	msolv = np.zeros(yy.size)

	for ii in range(yy.size):
		param_sm  = (pd_s[0], yy[ii], sigma, alpha0, phi, pd_s[1])
		m0        = .1
		msolv[ii] = max(0,bisect(medexp, a=0.0,b=0.9, args=param_sm))
	criterion = 0.0
	moms = [Psi1 - (pd_s[0]*np.mean(msolv))/(np.mean(yy)), 
		Psi2 - np.mean(1-np.exp(-pd_s[1]-alpha0*msolv))] 
	for m in moms:
		criterion += m**2
	return criterion

## Solution for the European countries

There was a mistake here as well as yy_eu was post-multiplied by y_eu again. 

In [ ]:
# Benchmark EU
#param_s     = (psi_eu[0], psi_eu[1], yy_eu, sigma, calibsol[1], #calibsol[0])
#pd_c0       = [0.8 , calibsol[2]]
#calibsol_eu = fsolve(hetero_calib_eu, pd_c0, args=param_s)#, full_output=True)
#calibsol_eu.x

In [ ]:
param_s     = (psi_eu[0], psi_eu[1], yy_eu, sigma, calibsol.x[1], calibsol.x[0])
pd_c0       = [0.9, calibsol.x[2]]
calibsol_eu = minimize(hetero_calib_eu_f, pd_c0, args=param_s,method='powell')#, full_output=True)
calibsol_eu.x

ValueError: f(a) and f(b) must have different signs

The price level makes a lot of sense and we also get better health...

## Individual decisions and health gradient: European countries

We now get this for gradients. 

In [ ]:
msolv = np.zeros(yy_us.size)

al0_calib = calibsol[0]
phi_calib = calibsol[1]
al1_calib = calibsol_eu[1]

for ii in range(yy_us.size):
    param_m   = (calibsol_eu[0], yy_eu[ii], sigma, calibsol[0], calibsol[1], calibsol_eu[1])
    m0        = .1
    msolv[ii] = max(0,fsolve(medexp, m0, args=param_m))

msolv, ( 1-np.exp(-(calibsol_eu[1]+calibsol[0]*msolv[-1])) )/(1-np.exp(-(calibsol_eu[1]+calibsol[0]*msolv[0])))

KeyError: 0

## Individual decisions and health gradient: the US

In [ ]:
msolv = np.zeros(yy_us.size)

al0_calib = calibsol[0]
phi_calib = calibsol[1]
al1_calib = calibsol[2]

for ii in range(yy_us.size):
    param_m   = (1, yy_us[ii], sigma, calibsol[0], calibsol[1], calibsol[2])
    m0        = .1
    msolv[ii] = max(0,fsolve(medexp, m0, args=param_m))

msolv, ( 1-np.exp(-(calibsol[2]+calibsol[0]*msolv[-1])) )/( 1-np.exp(-(calibsol[2]+calibsol[0]*msolv[0])) )

(array([0.        , 0.04787381, 0.15831734, 0.36980884]), 1.275999999981046)